<a href="https://colab.research.google.com/github/Raneemmohamed/1/blob/main/deeplearning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ranemelbasuony","key":"2416ec8bdcba2b2a3acfb5bac6a2af3e"}'}

In [5]:
import shutil
import os

os.makedirs("/root/.kaggle", exist_ok=True)

shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

os.chmod("/root/.kaggle/kaggle.json", 600)


In [6]:
!pip install kaggle


In [7]:
!kaggle datasets download -d datamunge/sign-language-mnist


Dataset URL: https://www.kaggle.com/datasets/datamunge/sign-language-mnist
License(s): CC0-1.0
 72% 45.0M/62.6M [00:00<00:00, 158MB/s]
100% 62.6M/62.6M [00:00<00:00, 174MB/s]


In [8]:
!unzip sign-language-mnist.zip


Archive:  sign-language-mnist.zip
  inflating: amer_sign2.png          
  inflating: amer_sign3.png          
  inflating: american_sign_language.PNG  
  inflating: sign_mnist_test.csv     
  inflating: sign_mnist_test/sign_mnist_test.csv  
  inflating: sign_mnist_train.csv    
  inflating: sign_mnist_train/sign_mnist_train.csv  


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_data = pd.read_csv("sign_mnist_train.csv")
test_data = pd.read_csv("sign_mnist_test.csv")

train_labels = train_data['label'].values
train_images = train_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1)

test_labels = test_data['label'].values
test_images = test_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1)

train_images = np.repeat(train_images, 3, axis=-1)
test_images = np.repeat(test_images, 3, axis=-1)

train_images = tf.image.resize(train_images, [64, 64])
test_images = tf.image.resize(test_images, [64, 64])

train_images, test_images = train_images / 255.0, test_images / 255.0

train_labels = keras.utils.to_categorical(train_labels, num_classes=25)
test_labels = keras.utils.to_categorical(test_labels, num_classes=25)

base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
base_model.trainable = False

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(25, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

model.save("fine_tuned_sign_language_model.h5")

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"\nTest Accuracy after Fine-Tuning: {test_acc:.4f}")

<ipython-input-9-5451b3c13836>:28: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 2, 2, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 25)                  │           3,225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,425,177 (9.25 MB)

 Trainable params: 167,193 (653.10 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 107s 118ms/step - accuracy: 0.2288 - loss: 2.7201 - val_accuracy: 0.6942 - val_loss: 1.2587
Epoch 2/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 140s 115ms/step - accuracy: 0.6915 - loss: 1.0913 - val_accuracy: 0.7995 - val_loss: 0.7943
Epoch 3/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 141s 114ms/step - accuracy: 0.8256 - loss: 0.6409 - val_accuracy: 0.8530 - val_loss: 0.5981
Epoch 4/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 143s 115ms/step - accuracy: 0.8855 - loss: 0.4414 - val_accuracy: 0.8755 - val_loss: 0.4894
Epoch 5/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 99s 116ms/step - accuracy: 0.9180 - loss: 0.3294 - val_accuracy: 0.8903 - val_loss: 0.4258
Epoch 1/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 167s 183ms/step - accuracy: 0.4328 - loss: 2.5701 - val_accuracy: 0.8127 - val_loss: 0.6190
Epoch 2/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 199s 180ms/step - accuracy: 0.7457 - loss: 0.8391 - val_accuracy: 0.8530 - val_loss: 0.4642
Epoch 3/5
858/858 ━━━━━━━━━━━━━━━━━━━━ 153s 178ms/step - accuracy: 0.8598 - loss: 0.

225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.9136 - loss: 0.2689

Test Accuracy after Fine-Tuning: 0.9184
